# Chapitre 2 : Correction d'images médicales

Ce notebook a pour objectif de vous montrer quelques exemples de méthodes de correction qu'on peut appliquer à des images issues de l'Imagerie par Résonnance Magnétique:
- Correction du biais des intensités
- Normalisation des intensités
- Filtrage

Pour pouvoir faire les exercices de ce notebook vous devez au préalable vérifier que les nouvelles librairies suivantes sont installées (en plus de celles déjà installées lors des séances précédentes) et les installer dans le cas contraire:
- `SimpleITK ` : bibliothèque pour le traitement d'images qui contient plusieurs fonctionalités compatibles avec différents formats d'images dont les formats d'imagerie médicale.

Les images de ce notebook proviennent du site: https://www.cancerimagingarchive.net/

## Exercice1: 

L'objectif de cet exercice est d'appliquer une correction du biais des intensités à une image IRM. 

1) En vous inspirant des codes développés dans le notebook 1, convertissez le volume dicom contenu dans le répertoire "t1_bias" au format nifti.

In [26]:
import dicom2nifti
dicom2nifti.convert_directory('./Données-20201001/t1_bias/t1_bias/', './', compression=True, reorient=True)

2) Créez un nouveau fichier python que vous appellerez "utils.py". Mettez dedans les fonctions de visualisation que vous avez développées lors de l'atelier précédent puis importez les fonctions explore_3dimage_axial, explore_3d_coronal et explore_3d_sagital.

In [27]:
from utils import explore_3dimage_sagital,explore_3dimage_coronal,explore_3dimage_axial

3) Affichez l'image t1_with_bias avec la fonction explore_3dimage_axial comme vous avez fait lors de la séance précédente.

In [49]:
import nibabel as nib
nii_volume = nib.load('./t1_axial.nii.gz')
volume_array = nii_volume.get_fdata()
interact(explore_3dimage_coronal, layer=(0, volume_array.shape[2] - 1),input_data=fixed(volume_array));

interactive(children=(IntSlider(value=11, description='layer', max=22), Output()), _dom_classes=('widget-inter…

4) Affichez l'image t1_with_bias cette fois-ci en vue coronale. Que remarquez vous? 

In [55]:
interact(explore_3dimage_sagital, layer=(0, volume_array.shape[0] - 1),input_data=fixed(volume_array)); 

interactive(children=(IntSlider(value=215, description='layer', max=431), Output()), _dom_classes=('widget-int…

5) Question bonus facultative: Que peut on faire pour avoir un affichage correct? Astuce: regarder certains paramètres caractéristiques de l'image dans l'entête de l'image nifti et modifier la fonction d'affichage.

In [59]:
print(nii_volume)

<class 'nibabel.nifti1.Nifti1Image'>
data shape (432, 512, 23)
affine: 
[[-4.28285420e-01 -3.35514434e-02  1.22731492e-01  1.00684189e+02]
 [-3.23687904e-02  4.25767571e-01  6.70486510e-01 -1.01201180e+02]
 [ 1.24584828e-02 -4.71978262e-02  5.96115446e+00 -5.57457275e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 432 512  23   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.         0.4296875  0.4296875  5.999998   1.         1.
  1.         1.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal

6) Lisez maintenant le fichier nifti "t1_with_bias.nii" en utilisant la librairie sitk et affichez sa taille. Comparez sa taille avec la taille du même fichier lu avec la librairie nibabel.

In [34]:
import nibabel as nib
import SimpleITK as sitk
import numpy as np
t1_fn = "./t1_axial.nii.gz" #The path to the .nii image
sitk_t1 = sitk.ReadImage(t1_fn) # Read the .nii image containing the volume with SimpleITK
t1 = sitk.GetArrayFromImage(sitk_t1) #Accessing the numpy array

In [35]:
print(t1.shape, nii_volume.shape)

(23, 512, 432) (432, 512, 23)


REMARQUE: J'ai remarqué que le fichier lu avec la librairie SimpleTIK est la transposée de celui lu avec la librairie nibabel

7) L'image que vous avez affichée en vue axiale est plus sombre dans la partie antérieure du cerveau par rapport à la partie postérieure. Cet effet est essentiellement dû aux inhomogéneités du champs magnétique. Il existe plusieurs méthodes de correction d'inhomogéneités du champs. Nous allons aujourd'hui tester celle disponible dans la librairie sitk. Utilisez la fonction sitk.N4BiasFieldCorrectionImageFilter() pour corriger l'image. 
Remarque: Pendant que le code tourne, lisez les articles suivants:
- https://www.nitrc.org/docman/view.php/6/880/sled.pdf
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3071855/

In [77]:
img = sitk.ReadImage(t1_fn)
img_mask = sitk.OtsuThreshold(img)
img = sitk.Cast(img, sitk.sitkFloat32)
corrector = sitk.N4BiasFieldCorrectionImageFilter()
img_c = corrector.Execute(img, img_mask)

8) Sauvegardez l'image corrigée du biais sous le nom "t1_bias_corrected.nii".

In [87]:
corr_array = sitk.GetArrayFromImage(img_c)
img_corr = nib.Nifti1Image(corr_array, np.eye(4))
img_corr.header.get_xyzt_units()
nib.save(img_corr,'./t1_bias_corrected.nii')


## Exercice2: 

L'objectif de cet exercice est d'appliquer une normalisation des intensités à une image IRM. 


1) Avec la librairie nibabel, lisez les trois volumes "flair_subject1.nii", "flair_subject2.nii" et "flair_subject3.nii".

In [58]:
import nibabel as nib
nii_volume_1 = nib.load("./Données-20201001/flair_subject1.nii")
print(nii_volume_1)
nii_volume_2 = nib.load("./Données-20201001/flair_subject2.nii")
print(nii_volume_2)
nii_volume_3 = nib.load("./Données-20201001/flair_subject3.nii")
print(nii_volume_3)

<class 'nibabel.nifti1.Nifti1Image'>
data shape (432, 512, 23)
affine: 
[[-4.23079938e-01 -6.58490211e-02 -5.03201306e-01  1.10255684e+02]
 [-5.11439927e-02  4.03860480e-01 -1.92030990e+00 -6.27262306e+01]
 [-5.49455918e-02  1.31118119e-01  5.66208649e+00 -1.18147316e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 432 512  23   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.         0.4296875  0.4296875  6.0000024  1.         1.
  1.         1.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal

2) Calculez et affichez les statistiques (min, max, mean, std) pour chacun des trois volumes. Que remarquez vous?

In [61]:
import SimpleITK as sitk
nii_volume = nib.load( './Données-20201001/flair_subject1.nii')
images_array = nii_volume.get_fdata()
print("Images Array Shape: ", images_array.shape)
print(' min: ',np.amin(images_array),' max: ',np.amax(images_array),' std: ' , np.std(images_array),' mean: ',np.mean(images_array))


Images Array Shape:  (432, 512, 23)
 min:  0.0  max:  1857.0  std:  176.03162654405423  mean:  161.92468183090529


In [62]:
nii_volume2 = nib.load( './Données-20201001/flair_subject2.nii')
images_array2 = nii_volume2.get_fdata()
print("Images Array Shape: ", images_array2.shape)
print(' min: ',np.amin(images_array2),' max: ',np.amax(images_array2),' std: ' , np.std(images_array2),' mean: ',np.mean(images_array2))

Images Array Shape:  (432, 512, 23)
 min:  0.0  max:  886.0  std:  132.72723658900023  mean:  119.59510810594053


In [63]:
nii_volume3 = nib.load( './Données-20201001/flair_subject3.nii')
images_array3 = nii_volume3.get_fdata()
print("Images Array Shape: ", images_array3.shape)
print(' min: ',np.amin(images_array3),' max: ',np.amax(images_array3),' std: ' , np.std(images_array3),' mean: ',np.mean(images_array3))

Images Array Shape:  (432, 512, 23)
 min:  0.0  max:  1753.0  std:  159.03284347872255  mean:  130.1007473219228


3) Ecrivez une fonction de normalisation des intensités de gris dans l'images que vous appellerez z_score_normalization. La fonction prend en entrée un numpy ndarray représentant les valeurs de gris du volume de données et renvoie un autre numpy nd_array contenant des valeurs de gris normalisées comme suit:

x_normalized = (x-mean)/std

In [44]:
def z_score_normalization(input_array):
    output_array  =(input_array - np.mean(input_array))/np.std(input_array)
    return output_array

4) Appliquez la fonction z_score_normalization à chacun des 3 volumes précédents et affichez leurs statistiques de nouveau:

In [64]:
norm_array  = z_score_normalization(images_array)
norm_array2  = z_score_normalization(images_array2)
norm_array3  = z_score_normalization(images_array3)

In [65]:
print(' min ',np.amin(norm_array),' max ',np.amax(norm_array),' mean ', np.mean(norm_array), ' std ', np.std(norm_array))

 min  -0.9198613056636246  max  9.629379398734805  mean  4.844934616319121e-17  std  1.0000000000000007


In [66]:
print(' min ',np.amin(norm_array2),' max ',np.amax(norm_array2),' mean ', np.mean(norm_array2), ' std ', np.std(norm_array2))

 min  -0.9010592790104994  max  5.774285004269992  mean  -1.1584935909132068e-16  std  0.9999999999999991


In [67]:
print(' min ',np.amin(norm_array3),' max ',np.amax(norm_array3),' mean ', np.mean(norm_array3), ' std ', np.std(norm_array3))

 min  -0.8180747100791752  max  10.204805606052119  mean  7.49087676840484e-17  std  1.0000000000000002


5) Question bonus (facultative): Enregistrez le volume normalisé obtenu à partir de l'image initiale "flair_subject1.nii" sous le nom "flair_subject1_normalized.nii"

In [74]:
img = nib.Nifti1Image(norm_array, np.eye(4))
img.header.get_xyzt_units()
nib.save(img,'./flair_subject1_normalized.nii')

In [76]:
flair_subject1_normalized = nib.load('./flair_subject1_normalized.nii')
print(flair_subject1_normalized)

<class 'nibabel.nifti1.Nifti1Image'>
data shape (432, 512, 23)
affine: 
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 432 512  23   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [1. 1. 1. 1. 1. 1. 1. 1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
